In [1]:
import sys
sys.path.append("C:\\Users\\sefi\\Meta-Modelling\\Sample Size vs Accuracy\\Noise Level 3\\Rastrigin10D\\")
import Utils
import pyDOE
import pandas as pd
import numpy as np
from RBFN import RBFN
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Helper Functions

In [2]:
''' RBF Network Interpolation for Robust Regularization'''
def RBF_minimax(train_data,test_data,Hyper):
    shape,sig = Hyper
    model = RBFN(hidden_shape=int(shape), sigma=sig)
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    model.fit(scaler.transform(np.array(train_data.iloc[:,:10])), np.array(train_data.iloc[:,11]))
    pred = model.predict(scaler.transform(np.array(test_data)))
    return model,pred

''' RBF Network Interpolation for Robust Composition'''
def RBF_composite(train_data,test_data,Hyper):
    shape,sig = Hyper
    model = RBFN(hidden_shape=int(shape), sigma=sig)
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    model.fit(scaler.transform(np.array(train_data.iloc[:,:10])), np.array(train_data.iloc[:,-1]))
    pred = model.predict(scaler.transform(np.array(test_data)))
    return model,pred

''' Latin HyperCube Sampling Design of Experiment for Hyper_Parameters Optimization'''
def DOE_Hyper():
    lhd = pyDOE.lhs(n=2, samples=500, criterion='m')
    X1, X2  = lhd[:,0], lhd[:,1] 
    X1 = Utils.linearscaletransform(X1,range_out=(5,1e2))
    X2 = Utils.linearscaletransform(X2,range_out=(1e-3 , 1e3))
    Hyper_Parameters = pd.DataFrame()
    Hyper_Parameters['Shape'] = pd.Series(X1)
    Hyper_Parameters['Sigma'] = pd.Series(X2)
    return Hyper_Parameters

## Load Training and Test Data Set initially Generated

In [3]:
12,17

(12, 17)

## RBFN

In [4]:
X = [50,100,150,200,250,300,350,400,450,500]
Kriging_minimax = np.zeros([len(X),2])
Kriging_composite = np.zeros([len(X),2])
for k in range(len(X)):
    print (X[k])
    train = pd.read_csv(sys.path[1][:-4]+str('Training_Data_Sets\\'+str(X[k])+'Samples.csv')).iloc[:,1:]
    test = pd.read_csv(sys.path[1][:-4]+str('Test_Data_Sets\\750Samples.csv')).iloc[:,1:]
    true_minimax = np.array(test.True_Minimax)
    true_composite = np.array(test.True_Composite)
    Hyper_Parameters = DOE_Hyper()
    mean_abs_error_minimax = np.zeros(Hyper_Parameters.shape[0])
    mean_abs_error_composite = np.zeros(Hyper_Parameters.shape[0])
    for i in range(Hyper_Parameters.shape[0]):
        temp = np.array(Hyper_Parameters.iloc[i,:])
        model_minimax,pred_k_m = RBF_minimax(train,test.iloc[:,:10],temp)
        model_composite,pred_k_c = RBF_composite(train,test.iloc[:,:10],temp)
        mean_abs_error_minimax[i] = np.mean((abs(true_minimax-pred_k_m) / abs(true_minimax) ) * 100)
        mean_abs_error_composite[i] = np.mean((abs( true_composite-pred_k_c) / abs(true_composite) ) * 100)
    Hyper_Parameters ['Mean_Error_Minimax'] = pd.Series(mean_abs_error_minimax)
    Hyper_Parameters ['Mean_Error_Composite'] = pd.Series(mean_abs_error_composite)
    Kriging_minimax [k,:] = np.array(Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Minimax.values.argmin(),:-2])
    Kriging_composite [k,:] = np.array(Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Composite.values.argmin(),:-2])

50
100
150
200
250
300
350
400
450
500


In [5]:
print ('Best Configuration of Hyper Parameters for Robust Regularization')
Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Minimax.values.argmin(),:-1]

Best Configuration of Hyper Parameters for Robust Regularization


Shape                 90.059425
Sigma                  0.001000
Mean_Error_Minimax     1.849618
Name: 313, dtype: float64

In [6]:
print ('Best Configuration of Hyper Parameters for Robust Composition')
Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Composite.values.argmin(),[0,1,3]]

Best Configuration of Hyper Parameters for Robust Composition


Shape                   90.059425
Sigma                    0.001000
Mean_Error_Composite     0.732452
Name: 313, dtype: float64

In [7]:
Hyper_Parameters.to_csv('Result_Hyper_Params.csv')

In [8]:
pd.DataFrame(Kriging_composite).to_csv('composite.csv')

In [9]:
pd.DataFrame(Kriging_composite).to_csv('composite.csv')
pd.DataFrame(Kriging_minimax).to_csv('minimax.csv')

In [10]:
minimax = pd.read_csv('minimax.csv').iloc[:,1:]
composite = pd.read_csv('composite.csv').iloc[:,1:]

In [11]:
np.array(minimax)

array([[7.09282034e+01, 1.00000000e-03],
       [7.96230039e+01, 1.00000000e-03],
       [4.56638968e+01, 1.00000000e-03],
       [9.42536820e+01, 1.00000000e-03],
       [7.15406807e+01, 1.00000000e-03],
       [7.32351977e+01, 1.00000000e-03],
       [6.87084920e+01, 1.00000000e-03],
       [8.21848068e+01, 1.00000000e-03],
       [2.82565359e+01, 1.00000000e-03],
       [9.00594246e+01, 1.00000000e-03]])

In [12]:
np.array(composite)

array([[7.09282034e+01, 1.00000000e-03],
       [7.96230039e+01, 1.00000000e-03],
       [4.56638968e+01, 1.00000000e-03],
       [9.42536820e+01, 1.00000000e-03],
       [7.15406807e+01, 1.00000000e-03],
       [7.32351977e+01, 1.00000000e-03],
       [6.87084920e+01, 1.00000000e-03],
       [8.21848068e+01, 1.00000000e-03],
       [2.82565359e+01, 1.00000000e-03],
       [9.00594246e+01, 1.00000000e-03]])